## Introduction to R logging

Before getting into anything too juicy, 
it is always nice to print the version of R first 
just so that we know that we are all on the same page 
and no one yells at anyone else on StackOverflow for any reason 
other than the fact that they just have a different version of R installed. 
If there was only ever one version of R...where all of the dependencies were installed.


In [1]:
cat(R.Version()$version.string)

R version 3.3.2 (2016-10-31)

In [2]:
# Load the library into R
# Dependencies: None, just a lot of patience.
library(logging)

#### The basic configurations
We can use the basic configuration command to set up a basic logging system for our use in R.
Our logger now has a handler and a level.
Levels are like layers or Russian dolls. When you have multiple loggers, loggers of a 'lower' level will not log other layers.


In [3]:
basicConfig()  # Generate the basic config
ls(getLogger())

[1] "addHandler"    "handlers"      "level"         "name"         
[5] "removeHandler" "setLevel"

In [4]:
# We see six attributes.
getLogger()[['level']]  # The level attribute shows what type of logger and the level.

INFO 
  20

There are five types of logs. All with different levels.
* logfinest &nbsp; &nbsp; ==  &nbsp; 1
* logdebug &nbsp; == &nbsp; 10
* loginfo  &nbsp; &nbsp; &nbsp; == &nbsp; 20
* logwarn  &nbsp; &nbsp; == &nbsp; 30
* logerror &nbsp; &nbsp; == &nbsp; 40

In [5]:
loginfo("Just letting you know")
logwarn("There's something I need to tell you.")
logdebug("I have issues.")
logerror("I can self-destruct")

2017-01-24 13:36:00 INFO::Just letting you know
2017-01-24 13:36:00 WARNING::There's something I need to tell you.
2017-01-24 13:36:00 ERROR::I can self-destruct


Notice how the debug one didn't print? That can be really useful to debug production scripts without anyone knowing (other than you) that there was a problem in the first place. But loggers shouldn't really be printing out anyway right? We should have a log directory in with separate warning, info, debug and error logs. So why are these ones going to stdout?

In [6]:
getLogger()[['handlers']]
getHandler('basic.stdout')[['level']]

$basic.stdout
<environment: 0x3758f58>


INFO 
  20

Ahh, that's why. 'basic.stdout' is of level info-20. This is why debug, at a lower level was not printed. So what can we do about this? We can add another handler instead. 

In [7]:
addHandler(writeToConsole)

In [8]:
with(getLogger(), names(handlers))

[1] "basic.stdout"   "writeToConsole"

Now we have two handlers, stdout and writeToConsole. There isn't much point in duplicating things, but there is use in writing warnings to stdout and general information to writeToConsole. Fortunately we can set levels on our handlers.

In [9]:
setLevel(30, getHandler('basic.stdout'))  # The warning level
setLevel(20, getHandler('writeToConsole'))  # Default level anyway.

In [10]:
loginfo('This is a piece of standard logging information')

2017-01-24 13:36:07 INFO::This is a piece of standard logging information


In [11]:
logwarn("This is a warning")

2017-01-24 13:36:07 WARNING::This is a warning
2017-01-24 13:36:07 WARNING::This is a warning


...so what happened here? Why has logwarn printed twice. Well because of the levels of the handlers. loginfo only printed to writeToConsole, its level is 20. logwarn printed to writeToConsole AND basic.stdout because of its higher rank.

In [12]:
addHandler(writeToFile, file="./error.log", level='ERROR')
# This handler will capture just the error outputs. Which is why you should make your
# errors very descriptive.

It is unfortunate that you cannot easily have two writeToFile handlers at different levels.
It might be handy to have something at info level writing to one file and error logs to another and warnings to a final log. Fortunately, we can create our own handlers and then set them up. For example the writeToFile console is just:

`writeToFile <- function(msg, handler){
    if (length(list(...)) && "dry" %in% names(list(...))) 
        return(exists("file", envir = handler))
    cat(paste(msg, "\n", sep = ""), file = with(handler, file), 
        append = TRUE)
}`


In [13]:
# Now let's write our own for debugging, info, warning and error.
writeToDebug <- function(msg, handler, ...)
{
   if (length(list(...)) && "dry" %in% names(list(...))) 
        return(exists("file", envir = handler))
    cat(paste(msg, "\n", sep = ""), file = with(handler, file), 
        append = TRUE)
}
writeToInfo <- function(msg, handler, ...)
{
   if (length(list(...)) && "dry" %in% names(list(...))) 
        return(exists("file", envir = handler))
    cat(paste(msg, "\n", sep = ""), file = with(handler, file), 
        append = TRUE)
}
writeToWarn <- function(msg, handler, ...)
{
   if (length(list(...)) && "dry" %in% names(list(...))) 
        return(exists("file", envir = handler))
    cat(paste(msg, "\n", sep = ""), file = with(handler, file), 
        append = TRUE)    
}
writeToError <- function(msg, handler, ...)
{
   if (length(list(...)) && "dry" %in% names(list(...))) 
        return(exists("file", envir = handler))
    cat(paste(msg, "\n", sep = ""), file = with(handler, file), 
        append = TRUE)
}

In [14]:
# Seems like quite a lot of code, not to mention setting the levels of each handler.
# Maybe we could turn this all into a function.
setLogger <- function(log_directory, prefix){
    dir.create(log_directory, showWarnings=FALSE)
    logReset()
    basicConfig()
    removeHandler('basic.stdout')  # Optionally keep it in and print out the info log.
    addHandler(writeToDebug, file=paste(log_directory,"/",prefix,".debug.log",sep=""),
               level='DEBUG')
    addHandler(writeToInfo, file=paste(log_directory,"/",prefix,".info.log",sep=""),
               level='INFO')
    addHandler(writeToWarn, file=paste(log_directory,"/",prefix,".warn.log",sep=""),
               level='WARN')    
    addHandler(writeToError, file=paste(log_directory,"/",prefix,".error.log",sep=""),
               level='ERROR')
}

In [15]:
setLogger("./log", "foo")

In [16]:
logdebug("This is a debug log")
loginfo("This is an info log")
logwarn("This is a warning log")
logerror("This is an error log")

So that seems pretty simple? Let's have a look at the output of each!

In [17]:
# Print contents of the debug file.
cat("\nPrinting the debugging log.\n")
cat(readLines("./log/foo.debug.log"), sep="\n")
# Print contents of the info file.
cat("\nPrinting the info log.\n")
cat(readLines("./log/foo.info.log"), sep="\n")
# Print contents of the warning file.
cat("\nPrinting the warning log.\n")
cat(readLines("./log/foo.warn.log"), sep="\n")
# Print contents of the error file.
cat("\nPrinting the error log.\n")
cat(readLines("./log/foo.error.log"), sep="\n")


Printing the debugging log.
2017-01-24 13:36:13 INFO::This is an info log
2017-01-24 13:36:13 WARNING::This is a warning log
2017-01-24 13:36:13 ERROR::This is an error log

Printing the info log.
2017-01-24 13:36:13 INFO::This is an info log
2017-01-24 13:36:13 WARNING::This is a warning log
2017-01-24 13:36:13 ERROR::This is an error log

Printing the warning log.
2017-01-24 13:36:13 WARNING::This is a warning log
2017-01-24 13:36:13 ERROR::This is an error log

Printing the error log.
2017-01-24 13:36:13 ERROR::This is an error log


We've covered everything that you really need to know.
The last thing to add is that we've been playing around with the 'default' logger the whole time. We can acutally name our loggers and point our logs/errors to a particular logger with a particular calibration. Loggers are only created when they are needed.

In [18]:
logReset()
addHandler(writeToConsole, logger='admin')
addHandler(writeToConsole, logger='peasants')
# From here we can direct logs in the script of the same level to different groups.
# Depending on the relevance to each group.
loginfo("We're running out of food.", logger='peasants')
loginfo("There appears to be a rebellion going on...", logger='admin')
logerror("")

2017-01-24 13:36:20 INFO:peasants:We're running out of food.
2017-01-24 13:36:20 INFO:admin:There appears to be a rebellion going on...


This is interesting, now our :: have been moved to :`<logger`>:

Any questions? My next talk will be about using Jupyter for Python and R. Because they're awesome!!